In [ ]:
!pip install GPyOpt
!pip install torch-pruning
!pip install paramz==0.9.6

In [ ]:
# Cell 0: Setup and Configuration Loading
# Purpose: Set up the environment and dynamically load user-specific model configuration and dataset from Notebook 3.

import torch
from torch.utils.data import DataLoader, Subset
import numpy as np
import pandas as pd
import os
import logging
import gc
import json
from google.colab import drive
import psutil  # For memory usage monitoring
from transformers import DistilBertTokenizer  # For fallback tokenization

# Ensure device is defined
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Mount Google Drive for loading configuration and dataset
drive.mount('/content/drive', force_remount=True)
drive_path = '/content/drive/MyDrive/Sentiment_Project'
os.makedirs(drive_path, exist_ok=True)

# Function to monitor and log memory usage
def log_memory_usage():
    process = psutil.Process()
    mem_info = process.memory_info()
    ram_usage_mb = mem_info.rss / 1024 ** 2
    logger.info(f"Current RAM usage: {ram_usage_mb:.2f} MB")
    if torch.cuda.is_available():
        gpu_mem = torch.cuda.memory_allocated() / 1024 ** 2
        logger.info(f"Current GPU memory usage: {gpu_mem:.2f} MB")
    return ram_usage_mb

# SentimentDataset definition (consistent with previous cells)
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        try:
            if input_ids is None or attention_mask is None or labels is None:
                raise ValueError("Input data (input_ids, attention_mask, labels) cannot be None")
            self.input_ids = torch.tensor(input_ids, dtype=torch.long)
            self.attention_mask = torch.tensor(attention_mask, dtype=torch.long)
            self.labels = torch.tensor(labels, dtype=torch.long)
            if len(self.input_ids) != len(self.attention_mask) or len(self.input_ids) != len(self.labels):
                raise ValueError(f"Length mismatch: input_ids ({len(self.input_ids)}), attention_mask ({len(self.attention_mask)}), labels ({len(self.labels)})")
            logger.info(f"SentimentDataset initialized with {len(self.labels)} samples")
        except Exception as e:
            logger.error(f"Failed to initialize SentimentDataset: {e}. Using empty dataset.")
            self.input_ids = torch.empty(0)
            self.attention_mask = torch.ones(0, 128)
            self.labels = torch.empty(0)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        try:
            return {'input_ids': self.input_ids[idx], 'attention_mask': self.attention_mask[idx], 'labels': self.labels[idx]}
        except Exception as e:
            logger.error(f"Error in __getitem__ at index {idx}: {e}")
            raise

# Load or regenerate dataset
input_ids, attention_mask, labels = None, None, None
dataset_path = os.path.join(drive_path, 'dataset_data.json')

if os.path.exists(dataset_path):
    try:
        with open(dataset_path, 'r') as f:
            data = json.load(f)
            input_ids = np.array(data['input_ids'])
            attention_mask = np.array(data['attention_mask'])
            labels = np.array(data['labels'])
        logger.info(f"Loaded dataset from {dataset_path} with {len(labels)} samples.")
    except (json.JSONDecodeError, KeyError, ValueError) as e:
        logger.error(f"Failed to load dataset from {dataset_path}: {e}. Attempting regeneration.")
else:
    logger.warning(f"Dataset file {dataset_path} not found. Attempting regeneration.")
    try:
        from google.colab import files  # For manual upload
        print("Dataset file not found. Please upload your dataset CSV (e.g., 'sentiment_data.csv') with 'review' and 'sentiment' columns.")
        uploaded = files.upload()
        import pandas as pd
        df = pd.read_csv(next(iter(uploaded)))
        tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        texts = df['review'].fillna('').tolist()
        encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='np')
        input_ids = encodings['input_ids']
        attention_mask = encodings['attention_mask']
        labels = df['sentiment'].map({'positive': 1, 'negative': 0, 'neutral': 2}).fillna(1).astype(int).values
        # Save regenerated data for future use
        with open(dataset_path, 'w') as f:
            json.dump({
                'input_ids': input_ids.tolist(),
                'attention_mask': attention_mask.tolist(),
                'labels': labels.tolist()
            }, f)
        logger.info(f"Regenerated and saved dataset to {dataset_path} with {len(labels)} samples.")
    except Exception as e:
        logger.error(f"Failed to regenerate dataset: {e}. Please run Notebook 3 (Cells 0-6) to prepare data or upload a valid CSV.")
        raise ValueError("Dataset regeneration failed. Upload a CSV with 'review' and 'sentiment' columns or run Notebook 3 and retry.")

# Load dataset into DataLoader
if 'loader' not in globals() or loader is None:
    try:
        logger.info("Loader not found. Recreating from input_ids, attention_mask, and labels.")
        dataset = SentimentDataset(input_ids, attention_mask, labels)
        if len(dataset) == 0:
            raise ValueError("Dataset is empty after initialization.")
        loader = DataLoader(dataset, batch_size=8, shuffle=True)
        logger.info(f"Recreated loader with {len(dataset)} samples.")
        first_batch = next(iter(loader), None)
        if first_batch is None:
            raise ValueError("Loader is empty; no batches available.")
        logger.info(f"First batch shapes: input_ids={first_batch['input_ids'].shape}, attention_mask={first_batch['attention_mask'].shape}, labels={first_batch['labels'].shape}")
    except Exception as e:
        logger.error(f"Failed to recreate loader: {e}. Please ensure valid data is available.")
        raise

# Dynamically load selected_config, best_accuracy, and target_accuracy from Notebook 3
output_path = os.path.join(drive_path, 'part3_output.json')
selected_config = None
best_accuracy = None
target_accuracy = None

if os.path.exists(output_path):
    try:
        with open(output_path, 'r') as f:
            part3_output = json.load(f)
            selected_config = part3_output['selected_config']
            best_accuracy = float(part3_output['best_accuracy'])
            target_accuracy = float(part3_output['target_accuracy'])
        logger.info(f"Loaded Part 3 output: {selected_config}, Best Accuracy={best_accuracy:.3f}, Target Accuracy={target_accuracy:.3f}")
    except (json.JSONDecodeError, KeyError, ValueError) as e:
        logger.error(f"Failed to load Part 3 output from {output_path}: {e}. Configuration required.")
else:
    logger.error(f"Part 3 output file {output_path} not found. Configuration required.")

# Generic fallback if no valid configuration is loaded
if selected_config is None or best_accuracy is None or target_accuracy is None:
    logger.warning("No valid configuration loaded from Notebook 3. Please run Notebook 3 first or provide configuration manually.")
    selected_config = {'approach': 'Unconfigured', 'message': 'Run Notebook 3 to select a model'}
    best_accuracy = 0.0
    target_accuracy = 0.920  # Retain target as a constant unless specified
    print("Warning: Model not configured. Please execute Notebook 3 (Sentiment_Analysis_Model_Optimization.ipynb) to select a model.")
    raise ValueError("Configuration missing. Please run Notebook 3 and retry.")

Mounted at /content/drive


In [ ]:
!ls "/content/drive/MyDrive/Sentiment_Project"

 coordinator_logs.txt
 Data_Analysis_Quality_Preprocessing.ipynb
 data_analysis_report.json
 dataset_data.json
 deploy
 explainability_output
 explainability_outputs
'fine_tuned_traditional_ml_(random_forest-like).joblib'
 hyperparams.json
 Notebook_5_CodeGen_Explainability.ipynb
 Part_1_Environment_Setup.ipynb
 Part_3_Model_Training_and_Evaluation.ipynb
 part3_output.json
 part4_output.json
 preprocessed_data.pt
 processed_dataset.csv
 quality_check_report.json
 selected_config_checkpoint.pkl
 Sentiment_Analysis_Model_Optimization.ipynb
 trained_traditional_ml_random_forest-like_encoder.joblib
 trained_traditional_ml_random_forest-like.joblib
 trained_traditional_ml_random_forest-like_vectorizer.joblib
'train_traditional_ml_(random_forest-like).py'
 train_traditional_ml_random_forest-like.py
 user_dataset_prompt.json
 user_feedback.csv


In [ ]:
# Cell 8a: Setup and Initial Checks
# Purpose: Set up the environment, define the device, and validate configuration.

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import os
import logging
import gc
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertForSequenceClassification, DistilBertForSequenceClassification
from tqdm import tqdm
import GPyOpt  # For Bayesian optimization
import torch_pruning as tp  # For model pruning
import torch.quantization  # For quantization
import csv
import psutil  # For memory usage monitoring (needed for log_memory_usage)

# Ensure device is defined (redundant but ensures standalone execution)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define drive path (assumes Cell 0 has mounted the drive)
drive_path = '/content/drive/MyDrive/Sentiment_Project'
os.makedirs(drive_path, exist_ok=True)

# Check configuration before proceeding
if 'selected_config' not in globals() or selected_config['approach'] == 'Unconfigured':
    logger.error("Model not configured. Skipping optimization and evaluation.")
    print("Error: Model not configured. Please run Notebook 3 and Cell 0 to set up the configuration.")
    raise ValueError("Configuration missing. Run Notebook 3 and Cell 0 first.")

# Check if loader exists (from Cell 0)
if 'loader' not in globals():
    logger.error("Data loader not found. Please run Cell 0 to initialize the dataset and loader.")
    raise ValueError("Data loader missing. Run Cell 0 first.")

# Log initial state
logger.info(f"Starting optimization and evaluation with approach: {selected_config['approach']}")
logger.info(f"Target Accuracy: {target_accuracy:.3f}, Best Accuracy so far: {best_accuracy:.3f}")

In [ ]:
# Cell 8b: Hyperparameter Optimization Using Bayesian Optimization
# Purpose: Dynamically optimize hyperparameters for any selected model and save results.

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import os
import pickle
import logging
import json
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from transformers import DistilBertForSequenceClassification, BertForSequenceClassification, RobertaForSequenceClassification, AlbertForSequenceClassification
from transformers import DistilBertTokenizer, BertTokenizer, RobertaTokenizer, AlbertTokenizer
import GPyOpt
from scipy.sparse import issparse
from datetime import datetime

# Configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DRIVE_PATH = '/content/drive/MyDrive/Sentiment_Project'
LOG_FORMAT = '%(asctime)s - %(levelname)s - %(message)s'
MIN_SAMPLES = 5  # Minimum samples for train-test split

# Logging Setup
logging.basicConfig(level=logging.INFO, format=LOG_FORMAT)
logger = logging.getLogger(__name__)

# Load Configuration
if 'selected_config' in globals():
    del selected_config
checkpoint_path = os.path.join(DRIVE_PATH, 'selected_config_checkpoint.pkl')
if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"Configuration file not found at {checkpoint_path}. Please run Notebook 3 first.")
with open(checkpoint_path, 'rb') as f:
    selected_config = pickle.load(f)
logger.info(f"Loaded initial configuration: {selected_config}")

# Expanded Model Registry
model_registry = {
    "Traditional ML (Random Forest-like)": {'class': RandomForestClassifier, 'type': 'Traditional ML'},
    "Traditional ML (Logistic Regression-like)": {'class': LogisticRegression, 'type': 'Traditional ML'},
    "Traditional ML (SVM-like)": {'class': lambda **params: SVC(probability=True, **params), 'type': 'Traditional ML'},
    "Traditional ML (Gradient Boosting-like)": {'class': GradientBoostingClassifier, 'type': 'Traditional ML'},
    "Traditional ML (Naive Bayes-like)": {'class': MultinomialNB, 'type': 'Traditional ML'},
    "Traditional ML (KNN-like)": {'class': KNeighborsClassifier, 'type': 'Traditional ML'},
    "Deep Learning (DistilBERT-like)": {
        'class': DistilBertForSequenceClassification, 'pretrained': 'distilbert-base-uncased', 'type': 'Deep Learning', 'tokenizer': DistilBertTokenizer
    },
    "Deep Learning (BERT-like)": {
        'class': BertForSequenceClassification, 'pretrained': 'bert-base-uncased', 'type': 'Deep Learning', 'tokenizer': BertTokenizer
    },
    "Deep Learning (RoBERTa-like)": {
        'class': RobertaForSequenceClassification, 'pretrained': 'roberta-base', 'type': 'Deep Learning', 'tokenizer': RobertaTokenizer
    },
    "Deep Learning (ALBERT-like)": {
        'class': AlbertForSequenceClassification, 'pretrained': 'albert-base-v2', 'type': 'Deep Learning', 'tokenizer': AlbertTokenizer
    },
    "Deep Learning (LSTM-like)": {'class': None, 'type': 'Deep Learning (RNN)', 'custom': True},
    "Deep Learning (CNN-like)": {'class': None, 'type': 'Deep Learning (CNN)', 'custom': True}
}

# Hyperparameter Spaces (Updated to Fix `learning_rate` Issue for RandomForestClassifier)
hyperparam_spaces = {
    "Traditional ML (Random Forest-like)": [
        {'name': 'n_estimators', 'type': 'discrete', 'domain': (50, 100, 200, 300)},
        {'name': 'max_depth', 'type': 'discrete', 'domain': (10, 20, 30, 50)},
        {'name': 'min_samples_split', 'type': 'discrete', 'domain': (2, 5, 10)}
    ],
    "Traditional ML (Logistic Regression-like)": [
        {'name': 'C', 'type': 'continuous', 'domain': (1e-4, 1e2)},
        {'name': 'max_iter', 'type': 'discrete', 'domain': (100, 500, 1000)}
    ],
    "Traditional ML (SVM-like)": [
        {'name': 'C', 'type': 'continuous', 'domain': (1e-4, 1e2)},
        {'name': 'kernel', 'type': 'categorical', 'domain': ('linear', 'rbf')}
    ],
    "Traditional ML (Gradient Boosting-like)": [
        {'name': 'n_estimators', 'type': 'discrete', 'domain': (50, 100, 200, 300)},
        {'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-3, 1.0)},
        {'name': 'max_depth', 'type': 'discrete', 'domain': (3, 5, 10)}
    ],
    "Traditional ML (Naive Bayes-like)": [
        {'name': 'alpha', 'type': 'continuous', 'domain': (1e-3, 1.0)}
    ],
    "Traditional ML (KNN-like)": [
        {'name': 'n_neighbors', 'type': 'discrete', 'domain': (3, 5, 7, 10)},
        {'name': 'weights', 'type': 'categorical', 'domain': ('uniform', 'distance')}
    ],
    "Deep Learning (DistilBERT-like)": [
        {'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-5, 5e-5)},
        {'name': 'batch_size', 'type': 'discrete', 'domain': (8, 16, 32)}
    ],
    "Deep Learning (BERT-like)": [
        {'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-5, 5e-5)},
        {'name': 'batch_size', 'type': 'discrete', 'domain': (8, 16, 32)}
    ],
    "Deep Learning (RoBERTa-like)": [
        {'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-5, 5e-5)},
        {'name': 'batch_size', 'type': 'discrete', 'domain': (8, 16, 32)}
    ],
    "Deep Learning (ALBERT-like)": [
        {'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-5, 5e-5)},
        {'name': 'batch_size', 'type': 'discrete', 'domain': (8, 16, 32)}
    ],
    "Deep Learning (LSTM-like)": [
        {'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-4, 1e-2)},
        {'name': 'batch_size', 'type': 'discrete', 'domain': (16, 32, 64)},
        {'name': 'hidden_size', 'type': 'discrete', 'domain': (64, 128, 256)}
    ],
    "Deep Learning (CNN-like)": [
        {'name': 'learning_rate', 'type': 'continuous', 'domain': (1e-4, 1e-2)},
        {'name': 'batch_size', 'type': 'discrete', 'domain': (16, 32, 64)},
        {'name': 'filters', 'type': 'discrete', 'domain': (32, 64, 128)}
    ]
}

# Valid Hyperparameters (Updated to Match `hyperparam_spaces`)
valid_hyperparams = {
    "Traditional ML (Random Forest-like)": {'n_estimators', 'max_depth', 'min_samples_split'},
    "Traditional ML (Logistic Regression-like)": {'C', 'max_iter'},
    "Traditional ML (SVM-like)": {'C', 'kernel'},
    "Traditional ML (Gradient Boosting-like)": {'n_estimators', 'learning_rate', 'max_depth'},
    "Traditional ML (Naive Bayes-like)": {'alpha'},
    "Traditional ML (KNN-like)": {'n_neighbors', 'weights'},
    "Deep Learning (DistilBERT-like)": {'learning_rate', 'batch_size'},
    "Deep Learning (BERT-like)": {'learning_rate', 'batch_size'},
    "Deep Learning (RoBERTa-like)": {'learning_rate', 'batch_size'},
    "Deep Learning (ALBERT-like)": {'learning_rate', 'batch_size'},
    "Deep Learning (LSTM-like)": {'learning_rate', 'batch_size', 'hidden_size'},
    "Deep Learning (CNN-like)": {'learning_rate', 'batch_size', 'filters'}
}

# Dynamic Model Selection
model_name = selected_config.get('model')
if not model_name or model_name not in model_registry:
    raise ValueError(f"Model '{model_name}' not specified or not supported. Supported models: {list(model_registry.keys())}")
model_info = model_registry[model_name]
model_type = model_info['type']
logger.info(f"Selected model: {model_name}, type: {model_type}")

# Data Preprocessing
selected_model = None
preprocessed_data = {'X': None, 'y': None, 'texts': None, 'full_X': None, 'full_y': None}
vectorizer = None
tokenizer = None

# Load Dataset
df_path = os.path.join(DRIVE_PATH, 'processed_dataset.csv')
if not os.path.exists(df_path):
    raise FileNotFoundError(f"Dataset not found at {df_path}")

df = pd.read_csv(df_path)
logger.info(f"Initial dataset shape: {df.shape}")
logger.info(f"Columns: {list(df.columns)}")
logger.info(f"Sample rows:\n{df.head().to_string()}")

# Dynamic Column Selection
feature_col = selected_config.get('feature_col', 'review')
label_col = selected_config.get('label_col', 'sentiment')

if feature_col not in df.columns or label_col not in df.columns:
    logger.warning(f"Specified columns (feature: {feature_col}, label: {label_col}) not found. Attempting to infer columns.")
    if len(df.columns) < 2:
        raise ValueError(f"Dataset must have at least 2 columns. Found: {list(df.columns)}")
    label_col = df.columns[-1]
    feature_col = df.columns[-2]
    logger.info(f"Inferred feature column: {feature_col}, label column: {label_col}")

# Data Cleaning
df = df.dropna(subset=[feature_col, label_col])
df = df[df[feature_col].astype(str).str.strip() != '']
df = df[df[label_col].astype(str).str.strip() != '']
logger.info(f"Shape after cleaning: {df.shape}")

# Dynamic Label Encoding
unique_labels = df[label_col].astype(str).str.lower().unique()
logger.info(f"Unique labels in '{label_col}': {unique_labels}")
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
logger.info(f"Label mapping: {label_mapping}")
if len(unique_labels) < 2:
    raise ValueError(f"Label column '{label_col}' must have at least 2 unique values. Found: {unique_labels}")

# Define Custom LSTM and CNN Models
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_ids, attention_mask=None):
        embedded = self.embedding(input_ids)
        lstm_out, _ = self.lstm(embedded)
        out = self.fc(lstm_out[:, -1, :])
        return out

class CNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, filters, output_size):
        super(CNNClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv = nn.Conv1d(embedding_dim, filters, kernel_size=3)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.fc = nn.Linear(filters, output_size)

    def forward(self, input_ids, attention_mask=None):
        embedded = self.embedding(input_ids).transpose(1, 2)
        conv_out = self.conv(embedded)
        pooled = self.pool(conv_out).squeeze(-1)
        out = self.fc(pooled)
        return out

# Data Preprocessing Based on Model Type
if model_type == 'Traditional ML':
    vectorizer = TfidfVectorizer(max_features=10000)
    full_X = vectorizer.fit_transform(df[feature_col].astype(str))
    full_y = df[label_col].astype(str).str.lower().map(label_mapping).astype(int)
elif model_type == 'Deep Learning':
    tokenizer_class = model_info.get('tokenizer', DistilBertTokenizer)
    tokenizer = tokenizer_class.from_pretrained(model_info['pretrained'])
    encodings = tokenizer(df[feature_col].astype(str).tolist(), truncation=True, padding=True, max_length=128, return_tensors='pt')
    full_X = {'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask']}
    full_y = torch.tensor(df[label_col].astype(str).str.lower().map(label_mapping).values, dtype=torch.long)
elif model_type in ['Deep Learning (RNN)', 'Deep Learning (CNN)']:
    from torchtext.vocab import build_vocab_from_iterator
    def yield_tokens(data_iter):
        for text in data_iter:
            yield text.lower().split()
    vocab = build_vocab_from_iterator(yield_tokens(df[feature_col].astype(str)), specials=['<unk>'])
    vocab.set_default_index(vocab['<unk>'])
    text_pipeline = lambda x: [vocab[token] for token in x.lower().split()]
    max_len = 128
    input_ids = []
    for text in df[feature_col].astype(str):
        tokens = text_pipeline(text)[:max_len]
        tokens += [0] * (max_len - len(tokens))
        input_ids.append(tokens)
    full_X = {'input_ids': torch.tensor(input_ids, dtype=torch.long)}
    full_y = torch.tensor(df[label_col].astype(str).str.lower().map(label_mapping).values, dtype=torch.long)
else:
    raise ValueError(f"Unsupported model type: {model_type}")

# Validate Data
if model_type == 'Traditional ML':
    if full_X.shape[0] != len(full_y):
        raise ValueError(f"Feature-label mismatch: {full_X.shape[0]} features vs {len(full_y)} labels")
else:
    if full_X['input_ids'].shape[0] != len(full_y):
        raise ValueError(f"Feature-label mismatch: {full_X['input_ids'].shape[0]} features vs {len(full_y)} labels")
if len(full_y) < 2:
    raise ValueError(f"Dataset too small: {len(full_y)} samples. Need at least 2 samples.")

# Check Label Distribution
label_counts = pd.Series(full_y.numpy() if torch.is_tensor(full_y) else full_y).value_counts()
logger.info(f"Label distribution: {label_counts.to_dict()}")

# Train-Test Split with Enhanced Robustness
use_full_dataset = False
if len(full_y) < MIN_SAMPLES:
    logger.warning(f"Dataset has {len(full_y)} samples, below minimum {MIN_SAMPLES}. Using full dataset without optimization.")
    use_full_dataset = True
else:
    test_size = 0.2
    min_train_samples = 4
    min_test_samples = 2
    required_samples = max(min_train_samples / (1 - test_size), min_test_samples / test_size)
    if len(full_y) < required_samples:
        logger.warning(f"Dataset too small for split: {len(full_y)} samples. Need {required_samples} samples. Using full dataset.")
        use_full_dataset = True
    else:
        # Ensure at least 2 samples per class in test set
        label_counts = pd.Series(full_y.numpy() if torch.is_tensor(full_y) else full_y).value_counts()
        min_class_samples = label_counts.min()
        test_class_samples = min_class_samples * test_size
        if test_class_samples < 2:
            logger.warning(f"Insufficient samples per class for stratified split: {test_class_samples} samples per class in test set. Using full dataset.")
            use_full_dataset = True
        else:
            if model_type == 'Traditional ML':
                X_train, X_test, y_train, y_test = train_test_split(full_X, full_y, test_size=test_size, random_state=42, stratify=full_y)
                logger.info(f"Split shapes: X_train={X_train.shape}, y_train={y_train.shape}, X_test={X_test.shape}, y_test={y_test.shape}")
                logger.info(f"y_train distribution: {pd.Series(y_train).value_counts().to_dict()}")
                logger.info(f"y_test distribution: {pd.Series(y_test).value_counts().to_dict()}")
            else:
                indices = np.arange(len(full_y))
                train_idx, test_idx, y_train, y_test = train_test_split(indices, full_y, test_size=test_size, random_state=42, stratify=full_y)
                X_train = {k: v[train_idx] for k, v in full_X.items()}
                X_test = {k: v[test_idx] for k, v in full_X.items()}
                logger.info(f"Split shapes: X_train input_ids={X_train['input_ids'].shape}, y_train={y_train.shape}, X_test input_ids={X_test['input_ids'].shape}, y_test={y_test.shape}")
                logger.info(f"y_train distribution: {pd.Series(y_train.numpy() if torch.is_tensor(y_train) else y_train).value_counts().to_dict()}")
                logger.info(f"y_test distribution: {pd.Series(y_test.numpy() if torch.is_tensor(y_test) else y_test).value_counts().to_dict()}")
            # Additional validation
            y_test_counts = pd.Series(y_test).value_counts()
            if len(y_test) < 2 or len(y_train) < 2 or any(y_test_counts < 2):
                logger.warning(f"Split produced insufficient samples: y_train={len(y_train)}, y_test={len(y_test)}, y_test distribution={y_test_counts.to_dict()}. Using full dataset.")
                use_full_dataset = True

if use_full_dataset:
    X_train, X_test, y_train, y_test = full_X, None, full_y, None
    logger.info(f"Using full dataset: X_train={'shape' in dir(X_train) and X_train.shape or X_train['input_ids'].shape}, y_train={y_train.shape}")

# Store Preprocessed Data
preprocessed_data['full_X'] = full_X
preprocessed_data['full_y'] = full_y
preprocessed_data['texts'] = df[feature_col].astype(str).tolist()
if not use_full_dataset:
    preprocessed_data['X'] = X_test
    preprocessed_data['y'] = y_test

# Bayesian Optimization
space = hyperparam_spaces.get(model_name, [])
optimal_params = []
if not space or use_full_dataset:
    logger.info(f"Optimization skipped due to {'no hyperparameter space' if not space else 'insufficient data'}. Using defaults.")
    if model_type == 'Traditional ML':
        default_params = {'n_estimators': 100, 'max_depth': 20, 'min_samples_split': 2} if 'Random Forest' in model_name else \
                         {'C': 1.0, 'max_iter': 100} if 'Logistic Regression' in model_name else \
                         {'C': 1.0, 'kernel': 'rbf'} if 'SVM' in model_name else \
                         {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3} if 'Gradient Boosting' in model_name else \
                         {'alpha': 1.0} if 'Naive Bayes' in model_name else \
                         {'n_neighbors': 5, 'weights': 'uniform'} if 'KNN' in model_name else {}
        selected_config['hyperparams'] = default_params
        optimal_params = list(default_params.values())
    else:
        default_params = {'learning_rate': 2e-5, 'batch_size': 16} if 'Deep Learning' in model_type else \
                         {'learning_rate': 1e-3, 'batch_size': 32, 'hidden_size': 128} if 'LSTM' in model_name else \
                         {'learning_rate': 1e-3, 'batch_size': 32, 'filters': 64}
        selected_config['hyperparams'] = default_params
        optimal_params = list(default_params.values())
    logger.info(f"Default hyperparameters: {selected_config['hyperparams']}")
else:
    def objective(params):
        params_dict = None
        try:
            params_dict = {space[i]['name']: params[i] for i in range(len(space))}
            # Validate hyperparameters against valid_hyperparams
            valid_params_set = valid_hyperparams.get(model_name, set())
            invalid_params = [k for k in params_dict.keys() if k not in valid_params_set]
            if invalid_params:
                logger.error(f"Invalid hyperparameters for {model_name}: {invalid_params}. Expected: {valid_params_set}")
                return 1.0  # Fallback score
            if model_type == 'Traditional ML':
                typed_params = {k: int(v) if k in {'n_estimators', 'max_depth', 'min_samples_split', 'max_iter', 'n_neighbors'} else v for k, v in params_dict.items()}
                model = model_info['class'](**typed_params)
                model.fit(X_train, y_train)
                if X_test is not None and y_test is not None:
                    logger.info(f"Before predict: X_test shape={'shape' in dir(X_test) and X_test.shape or X_test['input_ids'].shape}, y_test shape={y_test.shape}")
                    predictions = model.predict(X_test)
                    logger.info(f"After predict: predictions shape={predictions.shape}, y_test shape={y_test.shape}")
                    if len(predictions) != len(y_test):
                        logger.error(f"Prediction-label mismatch: {len(predictions)} predictions vs {len(y_test)} labels")
                        return 1.0  # Fallback score
                    if len(y_test) < 2:
                        logger.warning(f"y_test has insufficient samples: {len(y_test)}. Need at least 2. Returning default score.")
                        return 1.0  # Fallback score
                    y_test_counts = pd.Series(y_test).value_counts()
                    if any(y_test_counts < 2):
                        logger.warning(f"y_test has insufficient samples per class: {y_test_counts.to_dict()}. Returning default score.")
                        return 1.0  # Fallback score
                    accuracy = accuracy_score(y_test, predictions)
                    logger.debug(f"Accuracy: {accuracy} with params {typed_params}")
                    return -accuracy
                else:
                    predictions = model.predict(X_train)
                    if len(y_train) < 2:
                        logger.warning(f"y_train has insufficient samples: {len(y_train)}. Returning default score.")
                        return 1.0
                    accuracy = accuracy_score(y_train, predictions)
                    logger.debug(f"Fallback accuracy: {accuracy} with params {typed_params}")
                    return -accuracy
            else:
                typed_params = {k: int(v) if k in {'batch_size', 'hidden_size', 'filters'} else v for k, v in params_dict.items()}
                if model_type == 'Deep Learning':
                    model_class = model_info['class']
                    pretrained = model_info['pretrained']
                    model = model_class.from_pretrained(pretrained, num_labels=len(unique_labels)).to(DEVICE)
                elif model_type == 'Deep Learning (RNN)':
                    model = LSTMClassifier(vocab_size=len(vocab), embedding_dim=100, hidden_size=typed_params['hidden_size'], output_size=len(unique_labels)).to(DEVICE)
                else:
                    model = CNNClassifier(vocab_size=len(vocab), embedding_dim=100, filters=typed_params['filters'], output_size=len(unique_labels)).to(DEVICE)

                optimizer = optim.Adam(model.parameters(), lr=typed_params['learning_rate'])
                batch_size = typed_params['batch_size']
                dataset = TensorDataset(*[X_train[k] for k in X_train], y_train)
                temp_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
                model.train()
                for _ in range(1):
                    for batch in temp_loader:
                        inputs = {k: v.to(DEVICE) for k, v in zip(['input_ids', 'attention_mask'] if 'attention_mask' in X_train else ['input_ids'], batch[:-1])}
                        labels = batch[-1].to(DEVICE)
                        outputs = model(**inputs) if model_type == 'Deep Learning' else model(inputs['input_ids'])
                        logits = outputs.logits if model_type == 'Deep Learning' else outputs
                        loss = nn.CrossEntropyLoss()(logits, labels)
                        loss.backward()
                        optimizer.step()
                        optimizer.zero_grad()
                        torch.cuda.empty_cache()
                model.eval()
                predictions, true_labels = [], []
                if X_test is not None and y_test is not None:
                    test_dataset = TensorDataset(*[X_test[k] for k in X_test], y_test)
                    test_loader = DataLoader(test_dataset, batch_size=batch_size)
                else:
                    test_loader = temp_loader
                with torch.no_grad():
                    for batch in test_loader:
                        inputs = {k: v.to(DEVICE) for k, v in zip(['input_ids', 'attention_mask'] if 'attention_mask' in X_train else ['input_ids'], batch[:-1])}
                        labels = batch[-1].to(DEVICE)
                        outputs = model(**inputs) if model_type == 'Deep Learning' else model(inputs['input_ids'])
                        logits = outputs.logits if model_type == 'Deep Learning' else outputs
                        preds = torch.argmax(logits, dim=1)
                        predictions.extend(preds.cpu().numpy())
                        true_labels.extend(labels.cpu().numpy())
                if len(true_labels) < 2:
                    logger.warning(f"true_labels has insufficient samples: {len(true_labels)}. Returning default score.")
                    return 1.0
                true_labels_counts = pd.Series(true_labels).value_counts()
                if any(true_labels_counts < 2):
                    logger.warning(f"true_labels has insufficient samples per class: {true_labels_counts.to_dict()}. Returning default score.")
                    return 1.0
                accuracy = accuracy_score(true_labels, predictions)
                return -accuracy
        except Exception as e:
            logger.error(f"Objective failed with params {params_dict if params_dict else 'unknown'}: {str(e)}")
            return 1.0

    try:
        bo = GPyOpt.methods.BayesianOptimization(
            f=objective,
            domain=space,
            acquisition_type='EI',
            maximize=False,
            verbosity=True
        )
        bo.run_optimization(max_iter=20)
        optimized_params = {space[i]['name']: bo.x_opt[i] for i in range(len(space))}
        typed_params = {k: int(v) if k in {'n_estimators', 'max_depth', 'min_samples_split', 'max_iter', 'n_neighbors', 'batch_size', 'hidden_size', 'filters'} else v for k, v in optimized_params.items()}
        # Filter optimized_params to include only valid hyperparameters
        valid_params_set = valid_hyperparams.get(model_name, set())
        filtered_params = {k: v for k, v in typed_params.items() if k in valid_params_set}
        selected_config['hyperparams'] = filtered_params
        optimal_params = list(filtered_params.values())
        logger.info(f"Optimized hyperparameters (filtered): {filtered_params}")
    except Exception as e:
        logger.error(f"Optimization failed: {str(e)}")
        if model_type == 'Traditional ML':
            default_params = {'n_estimators': 100, 'max_depth': 20, 'min_samples_split': 2} if 'Random Forest' in model_name else \
                             {'C': 1.0, 'max_iter': 100} if 'Logistic Regression' in model_name else \
                             {'C': 1.0, 'kernel': 'rbf'} if 'SVM' in model_name else \
                             {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3} if 'Gradient Boosting' in model_name else \
                             {'alpha': 1.0} if 'Naive Bayes' in model_name else \
                             {'n_neighbors': 5, 'weights': 'uniform'} if 'KNN' in model_name else {}
            selected_config['hyperparams'] = default_params
            optimal_params = list(default_params.values())
        else:
            default_params = {'learning_rate': 2e-5, 'batch_size': 16} if 'Deep Learning' in model_type else \
                             {'learning_rate': 1e-3, 'batch_size': 32, 'hidden_size': 128} if 'LSTM' in model_name else \
                             {'learning_rate': 1e-3, 'batch_size': 32, 'filters': 64}
            selected_config['hyperparams'] = default_params
            optimal_params = list(default_params.values())
        logger.info(f"Using default hyperparameters: {selected_config['hyperparams']}")

# Model Initialization and Training
best_accuracy = None
try:
    if model_type == 'Traditional ML':
        if model_info['class'] is not None:
            typed_hyperparams = {k: int(v) if k in {'n_estimators', 'max_depth', 'min_samples_split', 'max_iter', 'n_neighbors'} else v for k, v in selected_config['hyperparams'].items()}
            selected_model = model_info['class'](**typed_hyperparams)
            selected_model.fit(preprocessed_data['full_X'], preprocessed_data['full_y'])
            if preprocessed_data['X'] is not None:
                predictions = selected_model.predict(preprocessed_data['X'])
                best_accuracy = accuracy_score(preprocessed_data['y'], predictions)
            else:
                predictions = selected_model.predict(preprocessed_data['full_X'])
                best_accuracy = accuracy_score(preprocessed_data['full_y'], predictions)
            logger.info(f"Model trained with hyperparameters: {typed_hyperparams}, Best Accuracy: {best_accuracy}")
        else:
            raise ValueError(f"Model {model_name} not implemented")
    else:
        typed_hyperparams = {k: int(v) if k in {'batch_size', 'hidden_size', 'filters'} else v for k, v in selected_config['hyperparams'].items()}
        if model_type == 'Deep Learning':
            model_class = model_info['class']
            pretrained = model_info['pretrained']
            selected_model = model_class.from_pretrained(pretrained, num_labels=len(unique_labels)).to(DEVICE)
        elif model_type == 'Deep Learning (RNN)':
            selected_model = LSTMClassifier(vocab_size=len(vocab), embedding_dim=100, hidden_size=typed_hyperparams['hidden_size'], output_size=len(unique_labels)).to(DEVICE)
        else:
            selected_model = CNNClassifier(vocab_size=len(vocab), embedding_dim=100, filters=typed_hyperparams['filters'], output_size=len(unique_labels)).to(DEVICE)

        optimizer = optim.Adam(selected_model.parameters(), lr=typed_hyperparams['learning_rate'])
        batch_size = typed_hyperparams['batch_size']
        dataset = TensorDataset(*[preprocessed_data['full_X'][k] for k in preprocessed_data['full_X']], preprocessed_data['full_y'])
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        selected_model.train()
        for _ in range(1):
            for batch in train_loader:
                inputs = {k: v.to(DEVICE) for k, v in zip(['input_ids', 'attention_mask'] if 'attention_mask' in preprocessed_data['full_X'] else ['input_ids'], batch[:-1])}
                labels = batch[-1].to(DEVICE)
                outputs = selected_model(**inputs) if model_type == 'Deep Learning' else selected_model(inputs['input_ids'])
                logits = outputs.logits if model_type == 'Deep Learning' else outputs
                loss = nn.CrossEntropyLoss()(logits, labels)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                torch.cuda.empty_cache()
        selected_model.eval()
        predictions, true_labels = [], []
        if preprocessed_data['X'] is not None:
            test_dataset = TensorDataset(*[preprocessed_data['X'][k] for k in preprocessed_data['X']], preprocessed_data['y'])
            test_loader = DataLoader(test_dataset, batch_size=batch_size)
        else:
            test_loader = train_loader
        with torch.no_grad():
            for batch in test_loader:
                inputs = {k: v.to(DEVICE) for k, v in zip(['input_ids', 'attention_mask'] if 'attention_mask' in preprocessed_data['X'] else ['input_ids'], batch[:-1])}
                labels = batch[-1].to(DEVICE)
                outputs = model(**inputs) if model_type == 'Deep Learning' else model(inputs['input_ids'])
                logits = outputs.logits if model_type == 'Deep Learning' else outputs
                preds = torch.argmax(logits, dim=1)
                predictions.extend(preds.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())
        best_accuracy = accuracy_score(true_labels, predictions)
        logger.info(f"Model trained with hyperparameters: {typed_hyperparams}, Best Accuracy: {best_accuracy}")
except Exception as e:
    logger.error(f"Model initialization failed: {str(e)}")
    best_accuracy = 0.0
    raise

# Convert NumPy Types to Python Types for JSON Serialization
def convert_to_json_serializable(obj):
    if isinstance(obj, (np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, (np.int32, np.int64)):
        return int(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_to_json_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_json_serializable(item) for item in obj]
    return obj

# Save Results
try:
    with open(os.path.join(DRIVE_PATH, 'selected_config_checkpoint.pkl'), 'wb') as f:
        pickle.dump(selected_config, f)
    logger.info("Configuration saved to checkpoint")

    part4_output = {
        "selected_config": convert_to_json_serializable(selected_config),
        "best_accuracy": float(best_accuracy),
        "target_accuracy": float(selected_config.get('target_accuracy', 0.92)),
        "timestamp": datetime.utcnow().isoformat() + "Z",
        "user_run_id": selected_config.get('user_run_id', '13735640')
    }
    with open(os.path.join(DRIVE_PATH, 'part4_output.json'), 'w') as f:
        json.dump(part4_output, f, indent=2)
    logger.info("Results saved to part4_output.json")
except Exception as e:
    logger.error(f"Failed to save results: {str(e)}")
    raise

# Cleanup
torch.cuda.empty_cache()
logger.info("Execution completed successfully")

ERROR:__main__:Objective failed with params unknown: index 1 is out of bounds for axis 0 with size 1
ERROR:__main__:Objective failed with params unknown: index 1 is out of bounds for axis 0 with size 1
ERROR:__main__:Objective failed with params unknown: index 1 is out of bounds for axis 0 with size 1
ERROR:__main__:Objective failed with params unknown: index 1 is out of bounds for axis 0 with size 1
ERROR:__main__:Objective failed with params unknown: index 1 is out of bounds for axis 0 with size 1
ERROR:__main__:Objective failed with params unknown: index 1 is out of bounds for axis 0 with size 1
ERROR:__main__:Objective failed with params unknown: index 1 is out of bounds for axis 0 with size 1
ERROR:__main__:Objective failed with params unknown: index 1 is out of bounds for axis 0 with size 1
ERROR:__main__:Objective failed with params unknown: index 1 is out of bounds for axis 0 with size 1
ERROR:__main__:Objective failed with params unknown: index 1 is out of bounds for axis 0 wi

In [ ]:
# Cell 8c: Apply Optimized Hyperparameters and Train Final Model
# Purpose: Use the optimized hyperparameters to train the final model.

# Ensure required imports are available
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import logging
from tqdm import tqdm

# Ensure logging is set up (if not already)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Ensure device is defined
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Validate optimal_params before applying
if 'optimal_params' not in globals():
    logger.error("optimal_params not defined. Ensure Cell 8b executed successfully.")
    raise ValueError("optimal_params must be defined from Cell 8b.")

# Apply optimization
if 'Traditional ML' in selected_config.get('approach', ''):
    # Validate optimal_params format (should align with hyperparam_spaces for the model)
    expected_params = len(valid_hyperparams.get(selected_config.get('model', ''), []))
    if not isinstance(optimal_params, (list, np.ndarray)) or len(optimal_params) != expected_params:
        logger.warning(f"Unexpected optimal_params format: {optimal_params}. Expected array with {expected_params} values. Using defaults.")
        optimal_params = [100, 10, 2] if 'Random Forest' in selected_config.get('model', '') else \
                         [1.0, 100] if 'Logistic Regression' in selected_config.get('model', '') else \
                         [1.0, 'rbf'] if 'SVM' in selected_config.get('model', '') else \
                         [100, 0.1, 3] if 'Gradient Boosting' in selected_config.get('model', '') else \
                         [1.0] if 'Naive Bayes' in selected_config.get('model', '') else \
                         [5, 'uniform'] if 'KNN' in selected_config.get('model', '') else [100, 10, 2]
    # Map optimal_params to the correct hyperparameter names
    hyperparam_names = list(valid_hyperparams.get(selected_config.get('model', ''), []))
    hyperparam_dict = {name: int(val) if name in {'n_estimators', 'max_depth', 'min_samples_split', 'max_iter', 'n_neighbors'} else val
                       for name, val in zip(hyperparam_names, optimal_params)}
    logger.info(f"Applying optimized hyperparameters: {hyperparam_dict}")
    try:
        selected_model.set_params(**hyperparam_dict)
    except Exception as e:
        logger.error(f"Failed to apply hyperparameters {hyperparam_dict}: {str(e)}. Using default parameters.")
        selected_model.set_params(n_estimators=100, max_depth=10, min_samples_split=2)

    logger.info("Training final model on full dataset with optimized hyperparameters...")
    selected_model.fit(preprocessed_data['full_X'], preprocessed_data['full_y'])
    logger.info("Final model training completed.")
else:  # Deep Learning
    # Validate optimal_params format (should be a list with 2 values for Deep Learning)
    expected_params = 2
    if not isinstance(optimal_params, (list, np.ndarray)) or len(optimal_params) != expected_params:
        logger.warning(f"Unexpected optimal_params format: {optimal_params}. Expected array with {expected_params} values. Using defaults.")
        optimal_params = [2e-5, 16]  # Default values for learning_rate, batch_size
    learning_rate, batch_size = optimal_params[0], int(optimal_params[1])
    optimizer = optim.AdamW(selected_model.parameters(), lr=learning_rate)
    loader = DataLoader(loader.dataset, batch_size=batch_size, shuffle=True)
    selected_model.train()
    for epoch in range(2):
        for batch in tqdm(loader, desc=f"Fine-Tuning {selected_config['approach']}"):
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = selected_model(**inputs, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            torch.cuda.empty_cache()

In [ ]:
# Cell 8c: Optimizer Agent - Population-Based Training (PBT)
# Purpose: Use PBT to dynamically adjust hyperparameters for deep learning models.

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import logging
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from transformers import (
    DistilBertForSequenceClassification, BertForSequenceClassification, RobertaForSequenceClassification
)

# Ensure device is defined
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure logging is set up
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Mapping of model names to their classes
model_mapping = {
    "Shallow Neural Network (MLP-like)": DistilBertForSequenceClassification,
    "Recurrent Neural Network (LSTM/GRU-like)": DistilBertForSequenceClassification,
    "Convolutional Neural Network (CNN-like)": DistilBertForSequenceClassification,
    "Bidirectional LSTM (BiLSTM-like)": BertForSequenceClassification,
    "Gated Recurrent Unit (GRU-like)": BertForSequenceClassification,
    "Feedforward Neural Network (FNN-like)": DistilBertForSequenceClassification,
    "Hybrid (CNN-RNN)": BertForSequenceClassification,
    "Deep Learning (Custom Transformer)": BertForSequenceClassification,
    "Deep Learning (Lightweight Pretrained Transformer, e.g., DistilBERT-like)": DistilBertForSequenceClassification,
    "BERT (Bidirectional Encoder Representations from Transformers)": BertForSequenceClassification,
    "RoBERTa (Robustly Optimized BERT Pretraining Approach)": RobertaForSequenceClassification,
    "ALBERT (A Lite BERT)": DistilBertForSequenceClassification,
    "XLNet (Generalized Autoregressive Pretraining)": BertForSequenceClassification,
    "T5 (Text-To-Text Transfer Transformer)": BertForSequenceClassification,
    "DeBERTa (Decoding-enhanced BERT with Disentangled Attention)": BertForSequenceClassification,
    "ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately)": BertForSequenceClassification,
    "Longformer (for long documents)": BertForSequenceClassification,
    "BigBird (sparse attention for long sequences)": BertForSequenceClassification,
    "Deep Learning (Advanced Pretrained Transformer, e.g., PaLM-like)": RobertaForSequenceClassification
}

# PBT training function
def pbt_training(model, loader, num_generations=3, population_size=3, initial_lr=None, initial_batch_size=None):
    # Validate inputs
    if model is None or not hasattr(model, 'train'):
        logger.error("Invalid model provided for PBT. Must be a trainable deep learning model.")
        raise ValueError("Model must be a valid deep learning model with train/eval methods.")
    if loader is None or not hasattr(loader, 'dataset'):
        logger.error("Invalid loader provided for PBT. Must be a valid DataLoader.")
        raise ValueError("Loader must be a valid DataLoader instance.")

    # Determine model class based on selected_config
    model_name = selected_config.get('model', 'Deep Learning (Lightweight Pretrained Transformer, e.g., DistilBERT-like)')
    model_class = model_mapping.get(model_name, DistilBertForSequenceClassification)
    pretrained_model = model_class.from_pretrained(model_class.pretrained if hasattr(model_class, 'pretrained') else 'distilbert-base-uncased', num_labels=2)

    # Initialize population with initial hyperparameters if provided (e.g., from Bayesian optimization)
    population = []
    for i in range(population_size):
        lr = initial_lr if initial_lr is not None else np.random.uniform(1e-5, 5e-5)
        batch_size = initial_batch_size if initial_batch_size is not None else np.random.choice([8, 16, 32])
        individual_model = pretrained_model if i == 0 else pretrained_model  # Reuse first model, clone others if needed
        population.append({
            'model': individual_model.to(device),
            'lr': lr,
            'batch_size': batch_size,
            'accuracy': 0.0
        })

    # PBT loop
    for generation in range(num_generations):
        logger.info(f"PBT Generation {generation + 1}")
        for individual in population:
            model = individual['model']
            optimizer = optim.AdamW(model.parameters(), lr=individual['lr'])
            temp_loader = DataLoader(loader.dataset, batch_size=individual['batch_size'], shuffle=True)
            model.train()
            try:
                for epoch in range(1):
                    for batch in temp_loader:
                        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
                        labels = batch['labels'].to(device)
                        outputs = model(**inputs, labels=labels)
                        loss = outputs.loss
                        loss.backward()
                        optimizer.step()
                        optimizer.zero_grad()
                        torch.cuda.empty_cache()
                # Evaluate
                model.eval()
                predictions, true_labels = [], []
                with torch.no_grad():
                    for batch in temp_loader:
                        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
                        labels = batch['labels'].to(device)
                        outputs = model(**inputs)
                        preds = torch.argmax(outputs.logits, dim=1)
                        predictions.extend(preds.cpu().numpy())
                        true_labels.extend(labels.cpu().numpy())
                individual['accuracy'] = accuracy_score(true_labels, predictions)
            except Exception as e:
                logger.error(f"Error during training/evaluation for generation {generation + 1}: {str(e)}")
                individual['accuracy'] = 0.0  # Assign low accuracy to handle failures

        # Exploit and explore
        population = sorted(population, key=lambda x: x['accuracy'], reverse=True)
        best_individual = population[0]
        for i in range(1, len(population)):
            if np.random.random() < 0.5:  # Exploit
                population[i]['lr'] = best_individual['lr']
                population[i]['batch_size'] = best_individual['batch_size']
            else:  # Explore
                population[i]['lr'] *= np.random.uniform(0.8, 1.2)
                population[i]['batch_size'] = np.random.choice([8, 16, 32])
                population[i]['model'] = pretrained_model.to(device)  # Reset model for exploration

        # Clear memory after each generation
        torch.cuda.empty_cache()

    best_individual = max(population, key=lambda x: x['accuracy'])
    logger.info(f"PBT Best Hyperparameters: lr={best_individual['lr']:.6f}, batch_size={best_individual['batch_size']}, accuracy={best_individual['accuracy']:.3f}")
    return best_individual['model'], best_individual['lr'], best_individual['batch_size']

# Apply PBT if deep learning model
if "Traditional ML" not in selected_config.get('approach', ''):
    try:
        # Use Bayesian-optimized hyperparameters as initial values if available
        initial_lr = None
        initial_batch_size = None
        if 'Deep Learning' in selected_config.get('approach', '') and 'optimal_params' in globals():
            initial_lr, initial_batch_size = optimal_params[0], int(optimal_params[1]) if len(optimal_params) >= 2 else (None, None)
            logger.info(f"Using Bayesian-optimized initial hyperparameters: lr={initial_lr}, batch_size={initial_batch_size}")

        # Validate selected_model before PBT
        if not hasattr(selected_model, 'train'):
            logger.warning("Selected model is not a deep learning model. Initializing new DistilBert model for PBT.")
            selected_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2).to(device)

        selected_model, learning_rate, batch_size = pbt_training(selected_model, loader, initial_lr=initial_lr, initial_batch_size=initial_batch_size)
        loader = DataLoader(loader.dataset, batch_size=batch_size, shuffle=True)
        logger.info("PBT completed. Updated loader with best batch_size.")
    except Exception as e:
        logger.error(f"PBT failed with error: {str(e)}")
        raise

In [ ]:
# Cell 8d: Optimizer Agent - Lightweight NAS (DARTS), Pruning, and Quantization
# Purpose: Optimize the deep learning model's architecture and efficiency.

import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import torch.quantization
import logging
from transformers import (
    DistilBertForSequenceClassification, BertForSequenceClassification, RobertaForSequenceClassification
)

# Ensure device is defined
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure logging is set up
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Mapping of model names to their classes for architecture validation
model_mapping = {
    "Shallow Neural Network (MLP-like)": DistilBertForSequenceClassification,
    "Recurrent Neural Network (LSTM/GRU-like)": DistilBertForSequenceClassification,
    "Convolutional Neural Network (CNN-like)": DistilBertForSequenceClassification,
    "Bidirectional LSTM (BiLSTM-like)": BertForSequenceClassification,
    "Gated Recurrent Unit (GRU-like)": BertForSequenceClassification,
    "Feedforward Neural Network (FNN-like)": DistilBertForSequenceClassification,
    "Hybrid (CNN-RNN)": BertForSequenceClassification,
    "Deep Learning (Custom Transformer)": BertForSequenceClassification,
    "Deep Learning (Lightweight Pretrained Transformer, e.g., DistilBERT-like)": DistilBertForSequenceClassification,
    "BERT (Bidirectional Encoder Representations from Transformers)": BertForSequenceClassification,
    "RoBERTa (Robustly Optimized BERT Pretraining Approach)": RobertaForSequenceClassification,
    "ALBERT (A Lite BERT)": DistilBertForSequenceClassification,
    "XLNet (Generalized Autoregressive Pretraining)": BertForSequenceClassification,
    "T5 (Text-To-Text Transfer Transformer)": BertForSequenceClassification,
    "DeBERTa (Decoding-enhanced BERT with Disentangled Attention)": BertForSequenceClassification,
    "ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately)": BertForSequenceClassification,
    "Longformer (for long documents)": BertForSequenceClassification,
    "BigBird (sparse attention for long sequences)": BertForSequenceClassification,
    "Deep Learning (Advanced Pretrained Transformer, e.g., PaLM-like)": RobertaForSequenceClassification
}

# Lightweight NAS with DARTS
def apply_darts(model):
    if not hasattr(model, 'eval') or not hasattr(model, 'train'):
        logger.error("Model is not a valid deep learning model for DARTS optimization.")
        raise ValueError("Model must be a trainable deep learning model.")

    model_name = selected_config.get('model', 'Deep Learning (Lightweight Pretrained Transformer, e.g., DistilBERT-like)')
    if isinstance(model, DistilBertForSequenceClassification):
        if hasattr(model.distilbert, 'transformer') and hasattr(model.distilbert.transformer, 'layer'):
            original_layers = len(model.distilbert.transformer.layer)
            model.distilbert.transformer.layer = nn.ModuleList(model.distilbert.transformer.layer[:3])  # Reduce to 3 layers
            logger.info(f"Applied DARTS: Reduced DistilBert layers from {original_layers} to 3.")
        else:
            logger.warning("DARTS not fully applicable to DistilBert model due to missing transformer layers.")
    elif isinstance(model, (BertForSequenceClassification, RobertaForSequenceClassification)):
        if hasattr(model.bert, 'encoder') and hasattr(model.bert.encoder, 'layer'):
            original_layers = len(model.bert.encoder.layer)
            model.bert.encoder.layer = nn.ModuleList(model.bert.encoder.layer[:3])  # Reduce to 3 layers
            logger.info(f"Applied DARTS: Reduced {model_name} layers from {original_layers} to 3.")
        else:
            logger.warning(f"DARTS not fully applicable to {model_name} due to missing encoder layers.")
    else:
        logger.warning(f"DARTS not implemented for model type: {model_name}")
    return model

# Pruning
def apply_pruning(model):
    if not hasattr(model, 'named_modules'):
        logger.error("Model is not a valid deep learning model for pruning.")
        raise ValueError("Model must be a valid deep learning model with named modules.")

    model_name = selected_config.get('model', 'Deep Learning (Lightweight Pretrained Transformer, e.g., DistilBERT-like)')
    if isinstance(model, (DistilBertForSequenceClassification, BertForSequenceClassification, RobertaForSequenceClassification)):
        for name, module in model.named_modules():
            if isinstance(module, nn.Linear):
                prune.l1_unstructured(module, name='weight', amount=0.3)  # Prune 30% of weights using L1 norm
        logger.info(f"Applied pruning to {model_name} model (30% of Linear layer weights).")
    else:
        logger.warning(f"Pruning not implemented for model type: {model_name}")
    return model

# Quantization
def apply_quantization(model):
    if not hasattr(model, 'eval'):
        logger.error("Model is not a valid deep learning model for quantization.")
        raise ValueError("Model must be a valid deep learning model with eval method.")

    model_name = selected_config.get('model', 'Deep Learning (Lightweight Pretrained Transformer, e.g., DistilBERT-like)')
    if isinstance(model, (DistilBertForSequenceClassification, BertForSequenceClassification, RobertaForSequenceClassification)):
        model.eval()  # Required for quantization
        try:
            model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)
            logger.info(f"Applied quantization to {model_name} model (dynamic qint8 on Linear layers).")
        except Exception as e:
            logger.error(f"Quantization failed: {str(e)}")
            raise
    else:
        logger.warning(f"Quantization not implemented for model type: {model_name}")
    return model

# Apply optimizations if deep learning model
if "Traditional ML" not in selected_config.get('approach', ''):
    try:
        # Validate selected_model
        if not hasattr(selected_model, 'eval') or not hasattr(selected_model, 'train'):
            logger.error("Selected model is not a valid deep learning model for optimization.")
            raise ValueError("Selected model must be a deep learning model with eval/train methods.")

        # Apply optimizations sequentially
        selected_model = apply_darts(selected_model)
        selected_model = apply_pruning(selected_model)
        selected_model = apply_quantization(selected_model)
        logger.info("All optimizations (DARTS, Pruning, Quantization) applied successfully.")
    except Exception as e:
        logger.error(f"Optimization failed with error: {str(e)}")
        raise

In [ ]:
# Cell 8e: Evaluator Agent - Fine-Tuned BERT for Evaluation and Retraining
# Purpose: Evaluate the optimized model using a fine-tuned BERT (for DL) or direct prediction (for ML) and retrain if needed.

import torch
import torch.nn as nn
import torch.optim as optim
import os
import logging
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset

# Ensure device is defined
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure logging is set up
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Define target_accuracy (example value; adjust as needed based on project requirements)
target_accuracy = 0.85  # Placeholder; replace with your desired threshold

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size  # Should be 30522 for bert-base-uncased

# Evaluator agent function
def evaluator_agent(model, loader, X=None, y=None):
    # Validate inputs based on model type
    if model is None:
        logger.error("Invalid model provided for evaluation. Model is None.")
        raise ValueError("Model must not be None.")
    if "Traditional ML" not in selected_config.get('approach', '') and not hasattr(model, 'eval'):
        logger.error("Invalid model provided for evaluation. Must be a valid deep learning model with eval method.")
        raise ValueError("Model must be a valid deep learning model with eval method.")
    if loader is None and ("Traditional ML" not in selected_config.get('approach', '')):
        logger.error("Invalid loader provided for evaluation. Must be a valid DataLoader for deep learning models.")
        raise ValueError("Loader must be a valid DataLoader instance for deep learning models.")

    # Debug: Inspect loader and dataset before reconstruction
    logger.info(f"Original Loader type: {type(loader)}")
    logger.info(f"Original Loader dataset type: {type(loader.dataset)}")
    try:
        sample_batch = next(iter(loader))
        logger.info(f"Original Sample batch keys: {list(sample_batch.keys())}")
        if 'input_ids' in sample_batch:
            logger.info(f"Original Sample input_ids shape: {sample_batch['input_ids'].shape}, min: {sample_batch['input_ids'].min().item()}, max: {sample_batch['input_ids'].max().item()}")
        if 'labels' in sample_batch:
            logger.info(f"Original Sample labels shape: {sample_batch['labels'].shape}, min: {sample_batch['labels'].min().item()}, max: {sample_batch['labels'].max().item()}")
    except Exception as e:
        logger.warning(f"Could not inspect original sample batch: {str(e)}")

    # Reconstruct loader for deep learning if necessary
    reconstructed_loader = loader  # Default to original loader
    if "Traditional ML" not in selected_config.get('approach', ''):
        # Check if loader provides BERT-compatible inputs
        sample_batch = next(iter(loader))
        if 'input_ids' not in sample_batch or 'attention_mask' not in sample_batch or sample_batch['input_ids'].max().item() >= vocab_size or sample_batch['input_ids'].min().item() < 0:
            logger.warning("Loader does not provide BERT-compatible inputs or contains invalid input_ids. Reconstructing loader...")
            # Extract raw text and labels from preprocessed_data or processed_dataset.csv
            texts = None
            labels = None
            if 'preprocessed_data' in globals():
                logger.info(f"preprocessed_data keys: {list(preprocessed_data.keys())}")
                possible_text_keys = ['texts', 'text', 'review', 'content', 'sentence']
                for key in possible_text_keys:
                    if key in preprocessed_data:
                        texts = preprocessed_data[key]
                        labels = preprocessed_data['y']
                        logger.info(f"Using preprocessed_data['{key}'] as text source.")
                        break
            if texts is None:
                df_path = os.path.join(drive_path, 'processed_dataset.csv')
                if not os.path.exists(df_path):
                    logger.error(f"Dataset CSV {df_path} not found. Ensure it exists in Google Drive.")
                    raise FileNotFoundError(f"Dataset CSV {df_path} not found.")
                df = pd.read_csv(df_path)
                for col in df.columns:
                    if col.lower() in possible_text_keys:
                        texts = df[col]
                        labels = df['sentiment'].map({'positive': 1, 'negative': 0, 'neutral': 2}).fillna(1).astype(int)
                        logger.info(f"Using df['{col}'] as text source from processed_dataset.csv.")
                        break
                if texts is None:
                    logger.error("No text column found in dataset. Expected one of: " + ", ".join(possible_text_keys))
                    raise ValueError("Dataset must contain a text column for tokenization.")
            # Tokenize the text
            encoded = tokenizer(texts.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')
            if encoded['input_ids'].max().item() >= vocab_size or encoded['input_ids'].min().item() < 0:
                logger.error(f"Tokenized input_ids out of range: min={encoded['input_ids'].min().item()}, max={encoded['input_ids'].max().item()}, vocab_size={vocab_size}")
                raise ValueError("Tokenized input_ids contain invalid indices.")
            dataset = TensorDataset(encoded['input_ids'], encoded['attention_mask'], torch.tensor(labels))
            reconstructed_loader = DataLoader(dataset, batch_size=loader.batch_size if hasattr(loader, 'batch_size') else 32, shuffle=True)
            logger.info("Reconstructed loader with BERT-compatible inputs.")
            # Debug: Inspect reconstructed loader
            sample_batch = next(iter(reconstructed_loader))
            logger.info(f"Reconstructed Sample batch keys: {list(sample_batch.keys())}")
            if 'input_ids' in sample_batch:
                logger.info(f"Reconstructed Sample input_ids shape: {sample_batch['input_ids'].shape}, min: {sample_batch['input_ids'].min().item()}, max: {sample_batch['input_ids'].max().item()}")

    # Load or fine-tune BERT evaluator (skip for Traditional ML)
    if "Traditional ML" not in selected_config.get('approach', ''):
        evaluator = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
        evaluator_path = os.path.join(drive_path, 'fine_tuned_bert_evaluator.pt')
        if os.path.exists(evaluator_path):
            try:
                evaluator.load_state_dict(torch.load(evaluator_path))
                logger.info("Loaded fine-tuned BERT evaluator from checkpoint.")
            except Exception as e:
                logger.error(f"Failed to load evaluator checkpoint: {str(e)}. Fine-tuning from scratch.")
        else:
            logger.info("No evaluator checkpoint found. Fine-tuning BERT evaluator...")
            optimizer = optim.AdamW(evaluator.parameters(), lr=2e-5)
            evaluator.train()
            try:
                for epoch in range(2):
                    for batch in tqdm(reconstructed_loader, desc="Fine-Tuning BERT Evaluator"):
                        # Validate input_ids range
                        if 'input_ids' in batch:
                            input_ids = batch['input_ids']
                            if input_ids.max().item() >= vocab_size or input_ids.min().item() < 0:
                                logger.error(f"Invalid input_ids in batch: min={input_ids.min().item()}, max={input_ids.max().item()}, vocab_size={vocab_size}")
                                raise ValueError("Batch input_ids contain invalid indices.")
                        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
                        labels = batch['labels'].to(device)
                        outputs = evaluator(**inputs, labels=labels)
                        loss = outputs.loss
                        loss.backward()
                        optimizer.step()
                        optimizer.zero_grad()
                        torch.cuda.empty_cache()
                torch.save(evaluator.state_dict(), evaluator_path)
                logger.info("Fine-tuned BERT evaluator saved to checkpoint.")
            except Exception as e:
                logger.error(f"Error during fine-tuning: {str(e)}")
                raise
    else:
        evaluator = None

    # Evaluate the optimized model
    predictions, true_labels = [], []
    if "Traditional ML" in selected_config.get('approach', ''):
        logger.info("Evaluating Traditional ML model with direct prediction.")
        if X is None or y is None:
            df_path = os.path.join(drive_path, 'processed_dataset.csv')
            if not os.path.exists(df_path):
                raise FileNotFoundError(f"Dataset CSV {df_path} not found. Ensure it exists in Google Drive.")
            df = pd.read_csv(df_path)
            if 'vectorizer' not in globals():
                logger.error("Vectorizer not available for Traditional ML evaluation. Ensure it was defined in Cell 8b.")
                raise ValueError("Vectorizer must be defined for Traditional ML evaluation.")
            X_test = vectorizer.transform(df['review'].fillna(''))
            predictions = model.predict(X_test)
            true_labels = y if y is not None else df['sentiment'].map({'positive': 1, 'negative': 0, 'neutral': 2}).fillna(1).astype(int)
            # Log prediction and label shapes for debugging
            logger.info(f"Traditional ML predictions shape: {np.array(predictions).shape}, true_labels shape: {np.array(true_labels).shape}")
        else:
            predictions = model.predict(X)
            true_labels = y
            logger.info(f"Traditional ML predictions shape: {np.array(predictions).shape}, true_labels shape: {np.array(true_labels).shape}")
    else:
        model.eval()
        with torch.no_grad():
            try:
                for batch in tqdm(reconstructed_loader, desc="Evaluating Optimized Model"):
                    # Validate input_ids range
                    if 'input_ids' in batch:
                        input_ids = batch['input_ids']
                        if input_ids.max().item() >= vocab_size or input_ids.min().item() < 0:
                            logger.error(f"Invalid input_ids in batch: min={input_ids.min().item()}, max={input_ids.max().item()}, vocab_size={vocab_size}")
                            raise ValueError("Batch input_ids contain invalid indices.")
                    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
                    labels = batch['labels'].to(device)
                    outputs = model(**inputs)
                    preds = torch.argmax(outputs.logits, dim=1) if hasattr(outputs, 'logits') else torch.zeros_like(labels)
                    predictions.extend(preds.cpu().numpy())
                    true_labels.extend(labels.cpu().numpy())
                    torch.cuda.empty_cache()
                # Log prediction and label shapes for debugging
                logger.info(f"Deep Learning predictions shape: {np.array(predictions).shape}, true_labels shape: {np.array(true_labels).shape}")
            except Exception as e:
                logger.error(f"Error during evaluation: {str(e)}")
                raise

    # Compute metrics
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average='weighted')
    logger.info(f"Evaluation Metrics: Accuracy={accuracy:.3f}, F1={f1:.3f}, Target={target_accuracy:.3f}")
    return accuracy, f1

# Evaluate the optimized model
try:
    # Use preprocessed_data from Cell 8b if available, otherwise set X and y to None
    X_eval = preprocessed_data['X'] if 'preprocessed_data' in globals() and 'X' in preprocessed_data else None
    y_eval = preprocessed_data['y'] if 'preprocessed_data' in globals() and 'y' in preprocessed_data else None
    accuracy, f1 = evaluator_agent(selected_model, loader, X_eval, y_eval)
except Exception as e:
    logger.error(f"Initial evaluation failed: {str(e)}")
    raise

# Trigger retraining if target not met
if accuracy < target_accuracy:
    logger.info("Target accuracy not met. Triggering retraining.")
    try:
        if "Traditional ML" in selected_config.get('approach', ''):
            if X_eval is None or y_eval is None:
                logger.error("X and y must be provided for Traditional ML retraining.")
                raise ValueError("X and y are required for Traditional ML retraining.")
            selected_model.fit(X_eval, y_eval) if hasattr(selected_model, 'fit') else logger.warning("No fit method for retraining.")
        else:
            selected_model.train()
            optimizer = optim.AdamW(selected_model.parameters(), lr=learning_rate if 'learning_rate' in globals() else 2e-5)
            for epoch in range(2):  # Increased epochs for better retraining
                for batch in tqdm(loader, desc="Retraining"):
                    # Validate input_ids range
                    if 'input_ids' in batch:
                        input_ids = batch['input_ids']
                        if input_ids.max().item() >= vocab_size or input_ids.min().item() < 0:
                            logger.error(f"Invalid input_ids in batch: min={input_ids.min().item()}, max={input_ids.max().item()}, vocab_size={vocab_size}")
                            raise ValueError("Batch input_ids contain invalid indices.")
                    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
                    labels = batch['labels'].to(device)
                    outputs = selected_model(**inputs, labels=labels)
                    loss = outputs.loss if hasattr(outputs, 'loss') else torch.tensor(0.0)
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                    torch.cuda.empty_cache()
        # Re-evaluate after retraining
        accuracy, f1 = evaluator_agent(selected_model, loader, X_eval, y_eval)
        logger.info(f"Post-retraining Metrics: Accuracy={accuracy:.3f}, F1={f1:.3f}")
    except Exception as e:
        logger.error(f"Retraining failed: {str(e)}")
        raise

In [ ]:
# Cell 8f: Feedback Collection and RL/MAML Reward Update
# Purpose: Collect user feedback and update RL/MAML rewards.

import os
import csv
import pandas as pd
import logging

# Ensure logging is set up (consistent with previous cells)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Verify drive_path exists
if 'drive_path' not in globals():
    logger.error("drive_path is not defined. Ensure it is set in an earlier cell (e.g., Cell 8a).")
    raise ValueError("drive_path must be defined.")

feedback_path = os.path.join(drive_path, 'user_feedback.csv')

# Collect feedback using input() since 'forms' module is unavailable
try:
    print("Please provide feedback on the model's performance:")
    satisfaction_input = input("How satisfied are you with the model's performance (1-5)? [default=3]: ") or "3"
    satisfaction = int(satisfaction_input)
    if not (1 <= satisfaction <= 5):
        logger.warning(f"Satisfaction score {satisfaction} is out of range (1-5). Clamping to nearest valid value.")
        satisfaction = max(1, min(5, satisfaction))

    comments = input("Any additional comments? [default='']: ") or ""
    logger.info(f"User Feedback: Satisfaction={satisfaction}, Comments={comments}")
except ValueError as e:
    logger.error(f"Invalid input for satisfaction score. Expected an integer between 1 and 5, got: {satisfaction_input}. Error: {str(e)}")
    raise ValueError("Satisfaction score must be an integer between 1 and 5.")
except Exception as e:
    logger.error(f"Error collecting feedback: {str(e)}")
    raise

# Save to CSV
try:
    file_exists = os.path.exists(feedback_path) and os.path.getsize(feedback_path) > 0
    with open(feedback_path, 'a', newline='') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(['Satisfaction', 'Comments', 'Timestamp'])
        writer.writerow([satisfaction, comments, pd.Timestamp.now()])
    logger.info(f"Feedback saved to {feedback_path}")
except Exception as e:
    logger.error(f"Error saving feedback to CSV: {str(e)}")
    raise

# Update RL/MAML Rewards (Placeholder for MAML integration)
def update_maml_rewards(satisfaction):
    reward = (satisfaction - 3) / 2  # Normalize to [-1, 1]
    logger.info(f"Updated MAML reward: {reward:.2f}")
    # Placeholder: In a real MAML setup, this reward would update the meta-learner's loss
    return reward

reward = update_maml_rewards(satisfaction)

Please provide feedback on the model's performance:
How satisfied are you with the model's performance (1-5)? [default=3]: 5
Any additional comments? [default='']: well


In [ ]:
# Cell 8g: Final Output and Cleanup
# Purpose: Display final metrics and clean up memory.

import torch
import gc
import logging
import psutil

# Ensure logging is set up (consistent with previous cells)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Function to monitor and log memory usage
def log_memory_usage():
    try:
        process = psutil.Process()
        mem_info = process.memory_info()
        ram_usage_mb = mem_info.rss / 1024 ** 2
        logger.info(f"Current RAM usage: {ram_usage_mb:.2f} MB")
        if torch.cuda.is_available():
            gpu_mem = torch.cuda.memory_allocated() / 1024 ** 2
            logger.info(f"Current GPU memory usage: {gpu_mem:.2f} MB")
        else:
            logger.info("CUDA not available; GPU memory usage not reported.")
        return ram_usage_mb
    except Exception as e:
        logger.error(f"Error logging memory usage: {str(e)}")
        return 0

# Display final metrics
try:
    print("\nFinal Model Configuration:")
    print(f"Approach: {selected_config['approach']}")
    print(f"Best Accuracy Achieved: {accuracy:.3f}")
    print(f"Target Accuracy: {target_accuracy:.3f}")
    print(f"Target Achieved: {accuracy >= target_accuracy}")
    print(f"F1 Score: {f1:.3f}")
except NameError as e:
    logger.error(f"Required variables (e.g., accuracy, f1, selected_config, target_accuracy) are not defined. Ensure Cell 8e executed successfully: {str(e)}")
    raise
except Exception as e:
    logger.error(f"Error displaying final metrics: {str(e)}")
    raise

# Memory Cleanup
try:
    torch.cuda.empty_cache()
    gc.collect()
    log_memory_usage()
except Exception as e:
    logger.error(f"Error during memory cleanup: {str(e)}")
    raise


Final Model Configuration:
Approach: Traditional ML
Best Accuracy Achieved: 0.743
Target Accuracy: 0.850
Target Achieved: False
F1 Score: 0.743
